In [50]:
import pandas as pd
import numpy as np
import re

### Load data frame from csv file

In [51]:
data_df = pd.read_csv('example_dataframe_100_recipes_0901')
data_df.head(10)

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,White Chocolate Macadamia Banana Bread,[],"['Baking', 'Milk, Eggs, Other Dairy', 'Produce...","['sugar', 'butter', 'ripe banana', 'milk', 're..."
1,Asian Chickpea Lettuce Wraps,['Asian'],"['Canned and Jarred', 'Ethnic Foods', 'Condime...","['canned chickpeas', 'chili sauce', 'barbecue ..."
2,Salsa Verde Chicken Tamales,['Mexican'],"['Meat', 'Meat', 'Produce', 'Produce', 'Spices...","['cooked chicken', 'whole chicken', 'garlic', ..."
3,Cheddar Polenta With Bacon Wrapped Asparagus,[],"['Produce', 'Meat', 'Canned and Jarred', 'Cere...","['asparagus', 'applewood smoked bacon', 'chick..."
4,Blasian's Deviled Eggs,['American'],"['Milk, Eggs, Other Dairy', 'Condiments', 'Con...","['hard boiled egg', 'mayonnaise', 'mustard', '..."
5,Sweet and Sticky Chicken Strips,[],"['Baking', 'Milk, Eggs, Other Dairy', 'Baking'...","['golden brown sugar', 'egg', 'wheat flour', '..."
6,Moist Zucchini Pineapple Sweetbread,[],"['Baking', 'Canned and Jarred', 'Spices and Se...","['low sodium baking powder', 'pineapple with j..."
7,Classic Eggs Benedict,['American'],"['Oil, Vinegar, Salad Dressing;Spices and Seas...","['hollandaise sauce', 'canadian bacon', 'engli..."
8,"Kale & chickpea stew with cumin, smoked paprik...",[],"['Produce', 'Canned and Jarred', 'Produce', 'O...","['kale', 'canned chickpeas', 'sweet potato', '..."
9,"Vanilla Cream Cakes, Easy and Fluffy Holiday C...",[],"['Refrigerated;Frozen', 'Milk, Eggs, Other Dai...","['puff pastry dough', 'egg', 'sugar', 'sugar',..."


### Define functions required to clean dataframe

In [52]:
def ingredients_cleanup (data_df, column):
    row_list_mexican = []
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        row = row.replace("[", '')
        row = row.replace("'", '')
        row = row.replace("]", '')
        row = row.replace('"', '')
        row_list_mexican = row.split( ',')
        data_df.loc[indexes[counter],column] = row_list_mexican
        counter += 1
    
    return data_df

In [53]:
def remove_leading_ws(data_df, column):
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        for i, element in enumerate(row):
            element = element.strip()
            data_df.loc[indexes[counter],column][i] = element
        counter += 1
    return data_df


In [54]:
def total_ing_list_from_df(data_df):
    composite_ingredients = []
    for row in data_df.ingredients_SP:
        for element in row:
            composite_ingredients.append(element)
    return composite_ingredients

In [55]:
def sort_and_count_ingredient_list(ingredient_list):
    ingredient_df = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.rename(columns = {0:  'ingredient'})
    ingredient_df['count'] = 1
    ingredient_counts = ingredient_df.groupby('ingredient').agg({'count':  'count'})
    ingredient_counts.sort_values('count', ascending=False, inplace=True)
    return ingredient_counts


In [56]:
def ingredient_replacement (data_df, replacement_dict):
    
    
    axises = data_df.index.values.tolist()
    counter = 0
    for row in data_df.ingredients_SP:
        for i, element in enumerate(row):
            if element in replacement_dict.keys():
                element = replacement_dict[element]
                data_df.loc[axises[counter],'ingredients_SP'][i] = element
        counter += 1
    return data_df

### Create a new data frame with only Mexican recipes

In [57]:
mexican_df = data_df[data_df['cuisine_SP'] == "['Mexican']"].copy()
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
2,Salsa Verde Chicken Tamales,['Mexican'],"['Meat', 'Meat', 'Produce', 'Produce', 'Spices...","['cooked chicken', 'whole chicken', 'garlic', ..."
58,Enchiladas Verdes (Green Enchiladas),['Mexican'],"['Meat', 'Produce;Ethnic Foods;Spices and Seas...","['boneless skinless chicken breast', 'dried ar..."
95,Authentic Mexican Wedding Cookies,['Mexican'],"['Milk, Eggs, Other Dairy', 'Baking', 'Baking'...","['butter', 'powdered sugar', 'vanilla extract'..."


### For each row (recipe) remove the unwanted characters from the ingredients string, split the string at commas to return a list, save the returned list back to the dataframe.  

In [58]:
mexican_df = ingredients_cleanup(mexican_df, 'ingredients_SP')
mexican_df = ingredients_cleanup(mexican_df, 'aisle_SP')
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
2,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[cooked chicken, whole chicken, garlic, oni..."
58,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[boneless skinless chicken breast, dried arbo..."
95,Authentic Mexican Wedding Cookies,['Mexican'],"[Milk, Eggs, Other Dairy, Baking, Baking, ...","[butter, powdered sugar, vanilla extract, w..."


### Remove leading white spaces from the ingredients_SP column

In [59]:
mexican_df = remove_leading_ws(mexican_df, 'ingredients_SP')

In [60]:
mexican_df.head()

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
2,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[cooked chicken, whole chicken, garlic, onion,..."
58,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[boneless skinless chicken breast, dried arbol..."
95,Authentic Mexican Wedding Cookies,['Mexican'],"[Milk, Eggs, Other Dairy, Baking, Baking, ...","[butter, powdered sugar, vanilla extract, whea..."


### Create a list of all ingredients from the cuisine 

In [61]:
composite_ingredients_mexican = total_ing_list_from_df(mexican_df)

### Create a dataframe from the composite ingredients list, group by ingredient name with count(), and sort descending

In [62]:
mexican_ing_groups = sort_and_count_ingredient_list(composite_ingredients_mexican)
mexican_ing_groups

,count
ingredient,
chili pepper,3
coarse sea salt,3
masa harina,2
wheat flour,2
water,2
salsa verde,2
boneless skinless chicken breast,1
salt,1
pecans,1


### Create a dictionary to use to replace different names for the same ingredients.  In final product create reg ex logic to make these changes

In [63]:
replacement_dict = {'coarse sea salt':  'salt_and_pepper',
                    'salsa verde':  'salsa',
                    'boneless skinless chicken breast':  'chicken',
                    'salt':  'salt and pepper',
                    'whole chicken':  'chicken',
                    'white onion':  'onion',
                    'solive oil':  'oil',
                    'palm oil':   'oil',
                    'dried arbol chile':  'chili pepper',
                    'cooking oil':  'oil',
                    'cooked chicken':  'chicken',
                    'yellow onion':  'onion'
}

### Use the replacement dictionary to replace the keys with the values

In [64]:
mexican_df = ingredient_replacement(mexican_df, replacement_dict)
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
2,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[chicken, chicken, garlic, onion, cumin, salt_..."
58,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[chicken, chili pepper, oil, buttermilk, wheat..."
95,Authentic Mexican Wedding Cookies,['Mexican'],"[Milk, Eggs, Other Dairy, Baking, Baking, ...","[butter, powdered sugar, vanilla extract, whea..."


### Create a new composite ingredient list from the data frame

In [65]:
composite_ingredients_mexican = total_ing_list_from_df(mexican_df)

### Create a dataframe from the composite ingredients list, group by ingredient name with count(), and sort descending

In [66]:
mexican_ing_groups = sort_and_count_ingredient_list(composite_ingredients_mexican)
mexican_ing_groups.head(20)

,count
ingredient,
chili pepper,4
onion,3
chicken,3
salt_and_pepper,3
oil,2
wheat flour,2
water,2
salsa,2
masa harina,2


In [67]:
for index, row in enumerate(mexican_ing_groups.index):
    if index < 20:
        mexican_df[row] = 0
mexican_df.columns   


Index(['recipe_name', 'cuisine_SP', 'aisle_SP', 'ingredients_SP',
       'chili pepper', 'onion', 'chicken', 'salt_and_pepper', 'oil',
       'wheat flour', 'water', 'salsa', 'masa harina', 'vanilla extract',
       'tomatillos', 'salt and pepper', 'queso fresco', 'powdered sugar',
       'pecans', 'butter', 'olive oil', 'buttermilk', 'mexican crema',
       'ground cinnamon'],
      dtype='object')

In [68]:
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP,chili pepper,onion,chicken,salt_and_pepper,oil,wheat flour,...,tomatillos,salt and pepper,queso fresco,powdered sugar,pecans,butter,olive oil,buttermilk,mexican crema,ground cinnamon
2,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[chicken, chicken, garlic, onion, cumin, salt_...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[chicken, chili pepper, oil, buttermilk, wheat...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,Authentic Mexican Wedding Cookies,['Mexican'],"[Milk, Eggs, Other Dairy, Baking, Baking, ...","[butter, powdered sugar, vanilla extract, whea...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
mexican_df.columns[4]

'chili pepper'

In [86]:
indexes = mexican_df.index.values.tolist()
for index in indexes:
    for col_name in mexican_df.columns:
        if col_name in mexican_df.ingredients_SP[index]:
            mexican_df.loc[index,col_name] = 1
mexican_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP,chili pepper,onion,chicken,salt_and_pepper,oil,wheat flour,...,tomatillos,salt and pepper,queso fresco,powdered sugar,pecans,butter,olive oil,buttermilk,mexican crema,ground cinnamon
2,Salsa Verde Chicken Tamales,['Mexican'],"[Meat, Meat, Produce, Produce, Spices and ...","[chicken, chicken, garlic, onion, cumin, salt_...",1,1,1,1,1,0,...,1,0,0,0,0,0,1,0,0,0
58,Enchiladas Verdes (Green Enchiladas),['Mexican'],"[Meat, Produce;Ethnic Foods;Spices and Season...","[chicken, chili pepper, oil, buttermilk, wheat...",1,1,1,0,1,1,...,0,1,1,0,0,0,0,1,1,0
95,Authentic Mexican Wedding Cookies,['Mexican'],"[Milk, Eggs, Other Dairy, Baking, Baking, ...","[butter, powdered sugar, vanilla extract, whea...",0,0,0,0,0,1,...,0,0,0,1,1,1,0,0,0,1


In [88]:
mexican_df.to_csv('Mexican_parsed_to_20_columns.csv')